In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
from tqdm.auto import tqdm
from segments import SegmentsClient

# Get data from root repository in segments, copy over to individual repositories

In [112]:
api_key = os.getenv("SEGMENTSAPI")
client = SegmentsClient(api_key)

In [118]:
# dataset_name="jmarkow/basler-nir-plexiglass-arena-keypoints-fused-round2"
root_dataset_name = "jmarkow/basler-nir-plexiglass-arena-keypoints-fused-kneejoints-reflectonly"
# samples = client.get_samples(dataset_name, label_status=["LABELED","REVIEWED"])
# labels = [client.get_label(_sample.uuid) for _sample in samples]
labelers = ["zeynep", "amartya", "claire", "brittany"]
labeler_username_map = {
    "zeynep": "eulutas",
    "amartya": "amartya",
    "claire": "cdeng",
    "brittany": "brittanyaddison"
}

In [119]:
dset_name = root_dataset_name.split("/")[1]

In [151]:
from segments.exceptions import AlreadyExistsError, CollaboratorError

In [155]:
force_delete = True  # BE CAREFUL WHEN IT'S SET TO TRUE!!!!

In [156]:
for _labeler in tqdm(labelers):
    new_dset_name = f"{dset_name}-{_labeler}"
    try:
        client.clone_dataset(root_dataset_name, new_dset_name, clone_labels=True)
    except AlreadyExistsError as e:
        if force_delete:
            client.delete_dataset(f"{root_dataset_name}-{_labeler}")
            client.clone_dataset(root_dataset_name, new_dset_name, clone_labels=True)
        else:
            pass

  0%|          | 0/4 [00:00<?, ?it/s]

In [157]:
import time

In [158]:
print("Pausing for 60 seconds...")
time.sleep(60)

Pausing for 60 seconds...


In [159]:
samples = {}
for _labeler in tqdm(labelers):
    samples[_labeler] = client.get_samples(f"{root_dataset_name}-{_labeler}", label_status=["LABELED","REVIEWED"])

  0%|          | 0/4 [00:00<?, ?it/s]

In [160]:
for _labeler, v in samples.items():
    print(_labeler)
    print(len(v))

zeynep
466
amartya
466
claire
466
brittany
466


In [161]:
# add everyone as a collaborator ONLY on their own dataset
for _labeler in tqdm(labelers):
    try:
        client.add_dataset_collaborator(f"{root_dataset_name}-{_labeler}", labeler_username_map[_labeler])
    except CollaboratorError as e:
        pass

  0%|          | 0/4 [00:00<?, ?it/s]

In [162]:
# get labels from each dataset and remove any non-labeler labels...

In [163]:
for _labeler, _samples in tqdm(samples.items()):
    labels = [client.get_label(_sample.uuid) for _sample in tqdm(_samples)]
    to_del = [
        (_sample.uuid, _label.labelset)
        for _sample, _label in zip(_samples, labels)
        if _label.created_by != labeler_username_map[_labeler]
    ]
    [client.delete_label(_uuid, _labelset) for (_uuid, _labelset) in tqdm(to_del)]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/466 [00:00<?, ?it/s]

  0%|          | 0/466 [00:00<?, ?it/s]

  0%|          | 0/466 [00:00<?, ?it/s]

  0%|          | 0/466 [00:00<?, ?it/s]

  0%|          | 0/466 [00:00<?, ?it/s]

  0%|          | 0/171 [00:00<?, ?it/s]

  0%|          | 0/466 [00:00<?, ?it/s]

  0%|          | 0/295 [00:00<?, ?it/s]